In [1]:
%%capture 
import os
import io
from firecloud import fiss
from pprint import pprint
import hail as hl
from hail.plot import output_notebook, show
import bokeh.io
from bokeh.io import *
from bokeh.resources import INLINE
bokeh.io.output_notebook(INLINE)

In [2]:
print(hl.citation())

Hail Team. Hail 0.2.130-bea04d9c79b5. https://github.com/hail-is/hail/releases/tag/0.2.130.


Start an Hail session

In [3]:
hl.init(default_reference = "GRCh37", log = 'MakingSense.log')
output_notebook()

/home/alberto/miniconda3/envs/mint/lib/python3.10/site-packages/hail/context.py:352: UserWarning:

Using hl.init with a default_reference argument is deprecated. To set a default reference genome after initializing hail, call `hl.default_reference` with an argument to set the default reference genome.



24/05/06 14:40:57 WARN Utils: Your hostname, alberto resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/05/06 14:40:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/05/06 14:40:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.4
SparkUI available at http://10.0.2.15:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.130-bea04d9c79b5
LOGGING: writing to MakingSense.log


Loading BokehJS ...

In [5]:
hl.utils.get_1kg('data/')

Print the VCF file:

In [6]:
vcf_files = [
    'data/*.vcf.bgz'
]

# Print a few of the paths to verify 
pprint(vcf_files)

['data/*.vcf.bgz']


Import the VCF into hail database:

In [7]:
mt = hl.import_vcf(vcf_files, force_bgz=True, array_elements_required=False, min_partitions=16).write("data/1kg.mt", overwrite=True)

Read indexed database --> faster!

In [8]:
mt = hl.read_matrix_table('data/1kg.mt')

Show samples in your dataset:

In [10]:
mt.s.show()

""
s
str
"""HG00096"""
"""HG00099"""
"""HG00105"""
"""HG00118"""
"""HG00129"""
"""HG00148"""
"""HG00177"""
"""HG00182"""


Describe you hail database:

In [11]:
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        BaseQRankSum: float64, 
        ClippingRankSum: float64, 
        DP: int32, 
        DS: bool, 
        FS: float64, 
        HaplotypeScore: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQ0: int32, 
        MQRankSum: float64, 
        QD: float64, 
        ReadPosRankSum: float64, 
        set: str
    }
----------------------------------------
Entry fields:
    'GT': call
    'AD': array<int32>
    'DP': int32
    'GQ': int32
    'PL': array<int32>
----------------------------------------
Colu

Count the occurrences in the hail database:

In [12]:
mt.count()

(10879, 284)

# Quality control on samples:

For the QC steps over samples, it's not necessary using all variants in the complete dataset.

The next passages will be in order:

1 - Filtering out samples with a genotyping rate < 90% and SNVs with a call rate < 90%

2 - Sex inference mismatches (MAF 10%)

4 - Filtering only SNVs, sex chromosomes and common SNVs (1%) not following HWE

5 - Calculating heterozygosity and removing deviating samples with F < -0.1 or > 0.1

6 - Remove high LD regions

7 - Filtering SNVs in LD

8 - Filtering related individual

9 - PCA population stratification

10 - Calculate SNVs, Indels, singletons for each sample

## Call rate

In [13]:
mt = hl.read_matrix_table("data/1kg.mt")

In [20]:
# run sample QC and save into matrix table
mt = hl.sample_qc(mt)

In [21]:
mt.sample_qc.show()

+-----------+-------------------------+--------------------------+
| s         | sample_qc.dp_stats.mean | sample_qc.dp_stats.stdev |
+-----------+-------------------------+--------------------------+
| str       |                 float64 |                  float64 |
+-----------+-------------------------+--------------------------+
| "HG00096" |                4.57e+00 |                 2.35e+00 |
| "HG00099" |                8.25e+00 |                 3.72e+00 |
| "HG00105" |                6.72e+00 |                 2.95e+00 |
| "HG00118" |                1.15e+01 |                 3.78e+00 |
| "HG00129" |                7.42e+00 |                 3.44e+00 |
| "HG00148" |                8.95e+00 |                 3.45e+00 |
| "HG00177" |                3.47e+00 |                 1.90e+00 |
| "HG00182" |                3.52e+00 |                 2.26e+00 |
| "HG00242" |                3.88e+00 |                 2.15e+00 |
| "HG00254" |                7.09e+00 |                 2.97e+00 |
+-----------+-------------------------+--------------------------+

+------------------------+------------------------+-------------------------+
| sample_qc.dp_stats.min | sample_qc.dp_stats.max | sample_qc.gq_stats.mean |
+------------------------+------------------------+-------------------------+
|                float64 |                float64 |                 float64 |
+------------------------+------------------------+-------------------------+
|               1.00e+00 |               2.00e+01 |                2.25e+01 |
|               1.00e+00 |               2.90e+01 |                3.76e+01 |
|               1.00e+00 |               2.20e+01 |                3.12e+01 |
|               0.00e+00 |               3.90e+01 |                4.88e+01 |
|               1.00e+00 |               3.20e+01 |                3.50e+01 |
|               0.00e+00 |               2.40e+01 |                3.97e+01 |
|               0.00e+00 |               1.50e+01 |                1.59e+01 |
|               1.00e+00 |               2.20e+01 |                1.54e+01 |
|               0.00e+00 |               1.80e+01 |                1.74e+01 |
|               1.00e+00 |               2.40e+01 |                3.27e+01 |
+------------------------+------------------------+-------------------------+

+--------------------------+------------------------+------------------------+
| sample_qc.gq_stats.stdev | sample_qc.gq_stats.min | sample_qc.gq_stats.max |
+--------------------------+------------------------+------------------------+
|                  float64 |                float64 |                float64 |
+--------------------------+------------------------+------------------------+
|                 2.25e+01 |               0.00e+00 |               9.90e+01 |
|                 2.81e+01 |               0.00e+00 |               9.90e+01 |
|                 2.58e+01 |               0.00e+00 |               9.90e+01 |
|                 2.79e+01 |               0.00e+00 |               9.90e+01 |
|                 2.78e+01 |               0.00e+00 |               9.90e+01 |
|                 2.79e+01 |               0.00e+00 |               9.90e+01 |
|                 1.69e+01 |               0.00e+00 |               9.90e+01 |
|                 1.67e+01 |               0.00e+00 |               9.90e+01 |
|                 1.73e+01 |               0.00e+00 |               9.90e+01 |
|                 2.55e+01 |               0.00e+00 |               9.90e+01 |
+--------------------------+------------------------+------------------------+

+---------------------+--------------------+------------------------+
| sample_qc.call_rate | sample_qc.n_called | sample_qc.n_not_called |
+---------------------+--------------------+------------------------+
|             float64 |              int64 |                  int64 |
+---------------------+--------------------+------------------------+
|            9.79e-01 |              10653 |                   

In [16]:
# export sample call rate:
mt.sample_qc.call_rate.export('data/sample_qc_data.tsv')

In [17]:
# filter for samples that are > 90% call rate
mt_cs = mt.filter_cols(mt.sample_qc.call_rate > 0.90)
mt_cs.count()

(10879, 281)

## task - gender

In [18]:
mt = hl.read_matrix_table("data/1kg.mt")

In [23]:
table = (hl.import_table('data/1kg_annotations.txt', impute=True)
         .key_by('Sample'))
table.show()

,,,,,
Sample,Population,SuperPopulation,isFemale,PurpleHair,CaffeineConsumption
str,str,str,bool,bool,int32
"""HG00096""","""GBR""","""EUR""",False,False,4
"""HG00097""","""GBR""","""EUR""",True,True,4
"""HG00098""","""GBR""","""EUR""",False,False,5
"""HG00099""","""GBR""","""EUR""",True,False,4
"""HG00100""","""GBR""","""EUR""",True,False,5
"""HG00101""","""GBR""","""EUR""",False,True,1
"""HG00102""","""GBR""","""EUR""",True,True,6
"""HG00103""","""GBR""","""EUR""",False,True,5


In [24]:
# Annotate matrix with phenotypes
mt = mt.annotate_cols(pheno = table[mt.s])

# Impute sex and remove sample mismatching
imputed_sex = hl.impute_sex(mt.GT)
imputed_sex[mt.s].f_stat.export('data/sex_data.tsv')
mt = mt.filter_cols(imputed_sex[mt.s].is_female != (mt.pheno.isFemale == True),keep=False)
mt.count()

(10879, 185)

## task - autosomes+common SNPs only+HWE

In [30]:
# read checkpoint database:
mt = hl.read_matrix_table('data/1kg.mt')
mt.count()

(10879, 284)

In [31]:
# For a better QC over samples, we are filtering only common SNPs:
aaf_threshold = 0.01
mt = mt.annotate_rows(aaf=hl.agg.call_stats(mt.GT, mt.alleles).AF[1])
mt = mt.filter_rows((mt.aaf > aaf_threshold) & (mt.aaf < (1 - aaf_threshold)))
mt.count()

(9127, 284)

In [32]:
# Filter to get SNVs only:
mt = hl.filter_alleles(mt, lambda allele, i: hl.is_snp(mt.alleles[0], allele))
mt = hl.sample_qc(mt)
mt = hl.variant_qc(mt)
mt.count()

(9127, 284)

In [33]:
mt.write('data/1kg_CKP1.mt', overwrite=True)

In [34]:
# Read ckp database:
mt = hl.read_matrix_table('data/1kg_CKP1.mt')

In [35]:
mt.count()

(9127, 284)

## task - het

In [38]:
# Read ckp database:
mt = hl.read_matrix_table('data/1kg_CKP1.mt')

In [39]:
#calculate heterozygosity using the n_het and n_called fields from the sample_qc method and fstatistic using the inbreeding method
mt=mt.annotate_cols(heterozygosity=((mt.sample_qc.n_het/mt.sample_qc.n_called)), inbreeding = hl.agg.inbreeding(mt.GT, mt.variant_qc.AF[1]))

In [ ]:
# #An alternative approach that filter out outlier based on a calculated threshold 

# #given the heterozygosity distribution, determine the min and the max heterozygosity values to be used when calculating a f-statistic cut off to filter samples

# het_stats=mt.aggregate_cols(hl.agg.stats(mt.heterozygosity))

# het_low=(het_stats.mean)-2.3* (het_stats.stdev)

# het_high=(het_stats.mean)+2.3 *(het_stats.stdev)

# het_data = mt.filter_cols((mt.heterozygosity <= het_high) & (mt.heterozygosity >= het_low))

# #calculate the fstatistic cut off

# het_fstat_cutoff = max(map(abs, het_data.inbreeding.f_stat.collect()))

# #filter samples filtered using fstatistic and sample call rate

# mt_filtered_fstat_scallRate = mt.filter_cols((mt.inbreeding.f_stat <= abs(het_fstat_cutoff_min)) &

# (mt.inbreeding.f_stat >= het_fstat_cutoff_min))

# mt_filtered_fstat_scallRate.count_cols()

In [40]:
mt.inbreeding.f_stat.export("data/het_data.tsv")

In [41]:
#Filter the mt based on QC cutoffs already established in literature (QC parkinson):
mt = mt.filter_cols((mt.inbreeding.f_stat > -0.1 ) & (mt.inbreeding.f_stat < 0.1 ))

In [42]:
mt.count()

(9127, 33)

## task - LD region

In [44]:
mt = hl.read_matrix_table('data/1kg_CKP1.mt')
mt.count()

(9127, 284)

In [46]:
# Load interval LD table
interval_LD_table = hl.import_bed('data/ld_regions.txt', reference_genome='GRCh37')
interval_LD_table.show()

""
interval
interval<locus<GRCh37>>
[1:48000001-1:52000001)
[2:86000001-2:100500001)
[2:134500001-2:138000001)
[2:183000001-2:190000001)
[3:47500001-3:50000001)
[3:83500001-3:87000001)
[3:89000001-3:97500001)
[5:44500001-5:50500001)


In [47]:
# Filter region in high LD prior PCA analysis
mt = mt.filter_rows(hl.is_defined(interval_LD_table[mt.locus]), keep=False)
mt.count()

(8807, 284)

In [48]:
mt.write('data/1kg_CKP2.mt', overwrite=True)

In [49]:
mt = hl.read_matrix_table('data/1kg_CKP2.mt')
mt.count()

(8807, 284)

## task - LD SNVs

In [50]:
mt = hl.read_matrix_table('data/1kg_CKP2.mt')
mt.count()

(8807, 284)

In [51]:
# Filter variants in LD
pruned_LD = hl.ld_prune(mt.GT, r2=0.2, bp_window_size=500000)
mt = mt.filter_rows(hl.is_defined(pruned_LD[mt.row_key]))
mt.write('data/1kg_CKP3.mt', overwrite=True)

In [52]:
mt = hl.read_matrix_table('data/1kg_CKP3.mt')
mt.count()

(8372, 284)

## task - IBD

In [53]:
mt = hl.read_matrix_table('data/1kg_CKP3.mt')
mt.count()

(8372, 284)

In [54]:
kin = hl.identity_by_descent(mt)

In [55]:
kin.export("data/ibd_data_pl.tsv")

In [56]:
# Calculate kinship matrix
pc_rel = hl.pc_relate(mt.GT, 0.01, k=2, statistics='kin')
pc_rel.export("data/ibd_data_pc_relate.tsv")
# Get samples with a kiship value over 0.125 (2nd degree)
pairs = pc_rel.filter(pc_rel['kin'] > 0.125)
pairs.show()
related_samples_to_remove = hl.maximal_independent_set(pairs.i, pairs.j, keep=False)
related_samples_to_remove.export("data/ibd_data_pc_relate_remove.tsv")

# Remove related individuals from dataset
mt = mt.filter_cols(hl.is_defined(related_samples_to_remove[mt.col_key]), keep=False)

,,
i,j,
s,s,kin
str,str,float64
"""HG01572""","""HG01991""",1.32e-01
"""HG01572""","""HG02259""",1.86e-01
"""HG01970""","""HG02259""",1.30e-01
"""HG01991""","""HG02259""",1.28e-01


## task - PCs

In [87]:
# Calculate PCs. The pca function produces eigenvalues as a list and sample PCs as a Table, and can also produce variant loadings when asked. 
# The hwe_normalized_pca function does the same, using HWE-normalized genotypes for the PCA. 
mt = hl.read_matrix_table('data/1kg.mt')
mt.count()

eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt.GT,compute_loadings=True)

pprint(eigenvalues)

[25.004388664119766,
 11.755847173122099,
 3.544215054905022,
 2.732334355335476,
 1.7214841939630867,
 1.6077925009566403,
 1.5607259207716793,
 1.547966747047974,
 1.5363332885508458,
 1.5121395254526566]


In [88]:
# Annotate columns with PCs scores
mt = mt.annotate_cols(scores = pcs[mt.s].scores)

In [89]:
mt.scores.export('data/PCA_scores.tsv')

In [92]:
table = (hl.import_table('data/1kg_annotations.txt', impute=True)
         .key_by('Sample'))
table.show()

# Annotate matrix with phenotypes
mt = mt.annotate_cols(pheno=table[mt.s])

,,,,,
Sample,Population,SuperPopulation,isFemale,PurpleHair,CaffeineConsumption
str,str,str,bool,bool,int32
"""HG00096""","""GBR""","""EUR""",False,False,4
"""HG00097""","""GBR""","""EUR""",True,True,4
"""HG00098""","""GBR""","""EUR""",False,False,5
"""HG00099""","""GBR""","""EUR""",True,False,4
"""HG00100""","""GBR""","""EUR""",True,False,5
"""HG00101""","""GBR""","""EUR""",False,True,1
"""HG00102""","""GBR""","""EUR""",True,True,6
"""HG00103""","""GBR""","""EUR""",False,True,5


In [93]:
p = hl.plot.scatter(mt.scores[0],
                    mt.scores[1],
                    label=mt.pheno.SuperPopulation,
                    title='PCA', xlabel='PC1', ylabel='PC2')
show(p)

/home/alberto/miniconda3/envs/mint/lib/python3.10/site-packages/hail/plot/plots.py:980: BokehDeprecationWarning:

'circle() method with size value' was deprecated in Bokeh 3.4.0 and will be removed, use 'scatter(size=...) instead' instead.

/home/alberto/miniconda3/envs/mint/lib/python3.10/site-packages/hail/plot/plots.py:980: BokehDeprecationWarning:

'circle() method with size value' was deprecated in Bokeh 3.4.0 and will be removed, use 'scatter(size=...) instead' instead.

/home/alberto/miniconda3/envs/mint/lib/python3.10/site-packages/hail/plot/plots.py:980: BokehDeprecationWarning:

'circle() method with size value' was deprecated in Bokeh 3.4.0 and will be removed, use 'scatter(size=...) instead' instead.

/home/alberto/miniconda3/envs/mint/lib/python3.10/site-packages/hail/plot/plots.py:980: BokehDeprecationWarning:

'circle() method with size value' was deprecated in Bokeh 3.4.0 and will be removed, use 'scatter(size=...) instead' instead.

/home/alberto/miniconda3/envs/mint/l

In [94]:
mt.write('data/1kg_PCA.mt', overwrite=True)

## task - indels+snvs etc metrics

Calculate SNVs, ratio INDELs, insertion, deletion, ratio titv for each samples.

In [60]:
# First, load the entire dataset and run sample QC:
mt = hl.read_matrix_table('data/1kg.mt')
mt = hl.sample_qc(mt)

In [61]:
# export metrix datsets:
mt.sample_qc.n_snp.export('data/snps_metrix.tsv')
mt.sample_qc.r_ti_tv.export('data/ti_tv_metrix.tsv')
mt.sample_qc.r_insertion_deletion.export('data/insertion_deletion_metrix.tsv')
mt.sample_qc.n_insertion.export('data/insertion_metrix.tsv')
mt.sample_qc.n_deletion.export('data/deletion_metrix.tsv')

In [62]:
# Calculate SNPs outliers using MAD, median absolute deviation:
metric_values = hl.agg.collect(mt.sample_qc.n_snp)
metric_median = hl.median(metric_values)
metric_mad = 1.4826 * hl.median(hl.abs(metric_values - metric_median))
outlier_metric=hl.struct( median=metric_median,
            mad=metric_mad,
            upper=metric_median + 4 * metric_mad,
            lower=metric_median - 4 * metric_mad)


mt = mt.annotate_globals(metrics_stats=mt.aggregate_cols(outlier_metric))

In [63]:
mt2=mt.filter_cols( (mt.sample_qc.n_snp <= mt.metrics_stats.upper) |
            (mt.sample_qc.n_snp >=  mt.metrics_stats.lower) )
mt2.count()

(10879, 284)

In [65]:
# Calculate ti/tv outliers using MAD, median absolute deviation:
metric_values = hl.agg.collect(mt.sample_qc.r_ti_tv)
metric_median = hl.median(metric_values)
metric_mad = 1.4826 * hl.median(hl.abs(metric_values - metric_median))
outlier_metric=hl.struct( median=metric_median,
            mad=metric_mad,
            upper=metric_median + 4 * metric_mad,
            lower=metric_median - 4 * metric_mad)


mt = mt.annotate_globals(metrics_stats=mt.aggregate_cols(outlier_metric))

In [66]:
mt2=mt.filter_cols( (mt.sample_qc.r_ti_tv <= mt.metrics_stats.upper) |
            (mt.sample_qc.r_ti_tv >=  mt.metrics_stats.lower) )
mt2.count()

(10879, 284)

# Variant QC

In [67]:
mt = hl.read_matrix_table('data/1kg.mt')
# Split multiallelic variant
mt = hl.split_multi_hts(mt)
# count
mt.count()

(10879, 284)

In [33]:
# remove sites non "." or "PASS":
# filter_condition_pass = (
#     hl.case()
#     .when(mt.GT.is_hom_ref(),
#           ( (hl.is_missing(mt.FT)) | (mt.FT=="PASS")
#           )
#          )
#     .when(mt.GT.is_het(),
#           ( (hl.is_missing(mt.FT)) | (mt.FT=="PASS")
#           )
#          )
#     .default( (hl.is_missing(mt.FT)) | (mt.FT=="PASS")
#             ) # hom-var
# )
#
# mt = mt.filter_entries(filter_condition_pass)

In [34]:
# ### Alberto's additional filters, uses AB version:
# AB = mt.AD[1] / hl.sum(mt.AD)

# filter_condition = (
#     hl.case()
#     .when(mt.GT.is_hom_ref(),
#           (((mt.GQ >= 20) & (hl.is_missing(mt.AD))) | 
#            ((mt.GQ >= 20) & ((hl.sum(mt.AD) >= 10) & (AB <= 0.1)))
#           )
#          )
#     .when(mt.GT.is_het(),
#           ( (mt.GQ >= 20) & ((hl.sum(mt.AD) >= 10) & ((AB > 0.2) & (AB < 0.8)))
#           )
#          )
#     .default((mt.GQ >= 20) & ((hl.sum(mt.AD) >= 10) & (AB >= 0.9))
#             ) # hom-var
# )

# mt = mt.filter_entries(filter_condition)

In [11]:
# Gnomad type:
# AB = mt.AD[1] / hl.sum(mt.AD)
#
# filter_condition = (
#     hl.case()
#     .when(mt.GT.is_hom_ref(),
#           (((mt.GQ >= 20) & (hl.is_missing(mt.AD))) |
#            ((mt.GQ >= 20) & ((hl.sum(mt.AD) >= 10)))
#           )
#          )
#     .when(mt.GT.is_het(),
#           ( (mt.GQ >= 20) & ((hl.sum(mt.AD) >= 10) & ((AB > 0.2)))
#           )
#          )
#     .default((mt.GQ >= 20) & ((hl.sum(mt.AD) >= 10))
#             ) # hom-var
# )
#
# mt = mt.filter_entries(filter_condition)

In [68]:
# variant qc:
mt = hl.variant_qc(mt)
# filter variant with a call rate upper than 90%:
mt = mt.filter_rows(mt.variant_qc.call_rate > 0.90)
# filter variants with AF==0 using allele frequency
mt = mt.filter_rows(mt.variant_qc.AF[1] == 0, keep=False)
# count
mt.count()

(10280, 284)

In [69]:
# # calculate HWE only on controls
# mt = mt.annotate_rows(hwe_ctrl = hl.agg.filter(mt.pheno.Status == "control", hl.agg.hardy_weinberg_test(mt.GT)))
# mt.row.show(100)

In [70]:
# # Filter based on HWE 10-5 in controls:
# mt = mt.filter_rows(mt.hwe_ctrl.p_value < 0.00001, keep=False)
# mt.count()

In [71]:
#mt.write('data/1kg_VARIANTQC.mt', overwrite=True)

# Annotation

In [72]:
# mt = hl.vep(mt)

In [ ]:
# db = hl.experimental.DB(region='us-central1', cloud='gcp')
# mt = db.annotate_rows_db(mt)

# Extraction

## examples with coordinates

In [73]:
mt = hl.read_matrix_table('data/1kg.mt')
mt.row.show()

+---------------+------------+------+----------+----------+--------------+
| locus         | alleles    | rsid |     qual | filters  | info.AC      |
+---------------+------------+------+----------+----------+--------------+
| locus<GRCh37> | array<str> | str  |  float64 | set<str> | array<int32> |
+---------------+------------+------+----------+----------+--------------+
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:909917      | ["G","A"]  | NA   | 1.58e+03 | NA       | [18]         |
| 1:986963      | ["C","T"]  | NA   | 3.98e+02 | NA       | [5]          |
| 1:1563691     | ["T","G"]  | NA   | 1.09e+03 | NA       | [64]         |
| 1:1707740     | ["T","G"]  | NA   | 9.35e+04 | NA       | [997]        |
| 1:2252970     | ["C","T"]  | NA   | 7.36e+02 | NA       | [6]          |
| 1:2284195     | ["T","C"]  | NA   | 1.42e+05 | NA       | [1559]       |
| 1:2779043     | ["T","C"]  | NA   | 2.89e+05 | NA       | [3532]       |
| 1:2944527     | ["G","A"]  | NA   | 1.24e+05 | NA       | [1206]       |
| 1:3761547     | ["C","A"]  | NA   | 1.61e+03 | NA       | [30]         |
+---------------+------------+------+----------+----------+--------------+

+----------------+---------+-------------------+----------------------+
| info.AF        | info.AN | info.BaseQRankSum | info.ClippingRankSum |
+----------------+---------+-------------------+----------------------+
| array<float64> |   int32 |           float64 |              float64 |
+----------------+---------+-------------------+----------------------+
| [1.03e-01]     |    5020 |         -3.39e+00 |            -1.70e-01 |
| [3.73e-03]     |    4830 |         -1.48e+00 |             1.26e-01 |
| [1.09e-03]     |    4588 |          1.25e+00 |            -3.77e+00 |
| [1.30e-02]     |    4766 |         -3.87e+01 |            -5.39e+00 |
| [1.98e-01]     |    5034 |         -4.04e+01 |            -2.87e-01 |
| [1.28e-03]     |    4682 |         -1.22e+00 |             1.79e+00 |
| [3.12e-01]     |    4990 |         -4.60e+01 |             3.50e-01 |
| [7.26e-01]     |    4866 |          1.74e+01 |             2.13e+00 |
| [2.45e-01]     |    4928 |          6.30e-02 |            -6.55e-01 |
| [5.95e-03]     |    5044 |         -4.47e+00 |            -8.82e+00 |
+----------------+---------+-------------------+----------------------+

+---------+---------+----------+---------------------+----------------------+
| info.DP | info.DS |  info.FS | info.HaplotypeScore | info.InbreedingCoeff |
+---------+---------+----------+---------------------+----------------------+
|   int32 |    bool |  float64 |             float64 |              float64 |
+---------+---------+----------+---------------------+----------------------+
|   17827 |   False | 2.23e+00 |                  NA |             9.88e-02 |
|   14671 |   False | 5.52e+00 |                  NA |            -5.00e-04 |
|   12398 |   False | 8.34e-01 |                  NA |             1.26e-02 |
|   15357 |   False | 1.90e+03 |                  NA |             2.70e-02 |
|   19902 |   False | 3.31e+00 |                  NA |             3.87e-02 |
|   14900 |   False | 2.82e+00 |                  NA |            -8.20e-03 |
|   18176 |   False | 2.95e+00 |                  NA |             9.25e-02 |
|   12878 |   False | 2.55e+01 |                  NA |             1.44e-01 |
|   17698 |   False | 4.49e-01 |                  NA |             1.23e-01 |
|   16845 |   False | 2.06e+00 |                  NA |            -4.70e-03 |
+---------+---------+----------+---------------------+----------------------+

+--------------+----------------+----------+----------+----------------+
| info.MLEAC   | info.MLEAF     |  info.MQ | info.MQ0 | info.MQRankSum |
+--------------+----------------+----------+----------+----------------+
| array<int32> | array<float64> |  float64 |    int32 |        float64 |
+--------------+----------------+----------+----------+----------------+
| [514]        | [1.02e-01]     

In [75]:
mt = hl.filter_intervals(mt, [hl.parse_locus_interval('1:1000-3761548')])
mt.count()

(10, 284)

In [76]:
mt.row.show(10)

+---------------+------------+------+----------+----------+--------------+
| locus         | alleles    | rsid |     qual | filters  | info.AC      |
+---------------+------------+------+----------+----------+--------------+
| locus<GRCh37> | array<str> | str  |  float64 | set<str> | array<int32> |
+---------------+------------+------+----------+----------+--------------+
| 1:904165      | ["G","A"]  | NA   | 5.23e+04 | NA       | [518]        |
| 1:909917      | ["G","A"]  | NA   | 1.58e+03 | NA       | [18]         |
| 1:986963      | ["C","T"]  | NA   | 3.98e+02 | NA       | [5]          |
| 1:1563691     | ["T","G"]  | NA   | 1.09e+03 | NA       | [64]         |
| 1:1707740     | ["T","G"]  | NA   | 9.35e+04 | NA       | [997]        |
| 1:2252970     | ["C","T"]  | NA   | 7.36e+02 | NA       | [6]          |
| 1:2284195     | ["T","C"]  | NA   | 1.42e+05 | NA       | [1559]       |
| 1:2779043     | ["T","C"]  | NA   | 2.89e+05 | NA       | [3532]       |
| 1:2944527     | ["G","A"]  | NA   | 1.24e+05 | NA       | [1206]       |
| 1:3761547     | ["C","A"]  | NA   | 1.61e+03 | NA       | [30]         |
+---------------+------------+------+----------+----------+--------------+

+----------------+---------+-------------------+----------------------+
| info.AF        | info.AN | info.BaseQRankSum | info.ClippingRankSum |
+----------------+---------+-------------------+----------------------+
| array<float64> |   int32 |           float64 |              float64 |
+----------------+---------+-------------------+----------------------+
| [1.03e-01]     |    5020 |         -3.39e+00 |            -1.70e-01 |
| [3.73e-03]     |    4830 |         -1.48e+00 |             1.26e-01 |
| [1.09e-03]     |    4588 |          1.25e+00 |            -3.77e+00 |
| [1.30e-02]     |    4766 |         -3.87e+01 |            -5.39e+00 |
| [1.98e-01]     |    5034 |         -4.04e+01 |            -2.87e-01 |
| [1.28e-03]     |    4682 |         -1.22e+00 |             1.79e+00 |
| [3.12e-01]     |    4990 |         -4.60e+01 |             3.50e-01 |
| [7.26e-01]     |    4866 |          1.74e+01 |             2.13e+00 |
| [2.45e-01]     |    4928 |          6.30e-02 |            -6.55e-01 |
| [5.95e-03]     |    5044 |         -4.47e+00 |            -8.82e+00 |
+----------------+---------+-------------------+----------------------+

+---------+---------+----------+---------------------+----------------------+
| info.DP | info.DS |  info.FS | info.HaplotypeScore | info.InbreedingCoeff |
+---------+---------+----------+---------------------+----------------------+
|   int32 |    bool |  float64 |             float64 |              float64 |
+---------+---------+----------+---------------------+----------------------+
|   17827 |   False | 2.23e+00 |                  NA |             9.88e-02 |
|   14671 |   False | 5.52e+00 |                  NA |            -5.00e-04 |
|   12398 |   False | 8.34e-01 |                  NA |             1.26e-02 |
|   15357 |   False | 1.90e+03 |                  NA |             2.70e-02 |
|   19902 |   False | 3.31e+00 |                  NA |             3.87e-02 |
|   14900 |   False | 2.82e+00 |                  NA |            -8.20e-03 |
|   18176 |   False | 2.95e+00 |                  NA |             9.25e-02 |
|   12878 |   False | 2.55e+01 |                  NA |             1.44e-01 |
|   17698 |   False | 4.49e-01 |                  NA |             1.23e-01 |
|   16845 |   False | 2.06e+00 |                  NA |            -4.70e-03 |
+---------+---------+----------+---------------------+----------------------+

+--------------+----------------+----------+----------+----------------+
| info.MLEAC   | info.MLEAF     |  info.MQ | info.MQ0 | info.MQRankSum |
+--------------+----------------+----------+----------+----------------+
| array<int32> | array<float64> |  float64 |    int32 |        float64 |
+--------------+----------------+----------+----------+----------------+
| [514]        | [1.02e-01]     

# Analysis

## GWAS

In [80]:
mt = hl.read_matrix_table('data/1kg.mt')
mt = hl.variant_qc(mt)
mt.count()

(10879, 284)

In [81]:
mt = mt.filter_rows(mt.variant_qc.AF[1] > 0.01)
mt = mt.filter_rows(mt.variant_qc.p_value_hwe > 1e-6)
print('Samples: %d  Variants: %d' % (mt.count_cols(), mt.count_rows()))

Samples: 284  Variants: 7912


In [83]:
table = (hl.import_table('data/1kg_annotations.txt', impute=True)
         .key_by('Sample'))
mt = mt.annotate_cols(pheno = table[mt.s])

In [84]:
gwas = hl.linear_regression_rows(y=mt.pheno.CaffeineConsumption,
                                 x=mt.GT.n_alt_alleles(),
                                 covariates=[1.0])
gwas.row.describe()

--------------------------------------------------------
Type:
        struct {
        locus: locus<GRCh37>, 
        alleles: array<str>, 
        n: int32, 
        sum_x: float64, 
        y_transpose_x: float64, 
        beta: float64, 
        standard_error: float64, 
        t_stat: float64, 
        p_value: float64
    }
--------------------------------------------------------
Source:
Index:
    ['row']
--------------------------------------------------------


In [85]:
p = hl.plot.manhattan(gwas.p_value)
show(p)

/home/alberto/miniconda3/envs/mint/lib/python3.10/site-packages/hail/plot/plots.py:980: BokehDeprecationWarning:

'circle() method with size value' was deprecated in Bokeh 3.4.0 and will be removed, use 'scatter(size=...) instead' instead.

/home/alberto/miniconda3/envs/mint/lib/python3.10/site-packages/hail/plot/plots.py:980: BokehDeprecationWarning:

'circle() method with size value' was deprecated in Bokeh 3.4.0 and will be removed, use 'scatter(size=...) instead' instead.

/home/alberto/miniconda3/envs/mint/lib/python3.10/site-packages/hail/plot/plots.py:980: BokehDeprecationWarning:

'circle() method with size value' was deprecated in Bokeh 3.4.0 and will be removed, use 'scatter(size=...) instead' instead.

/home/alberto/miniconda3/envs/mint/lib/python3.10/site-packages/hail/plot/plots.py:980: BokehDeprecationWarning:

'circle() method with size value' was deprecated in Bokeh 3.4.0 and will be removed, use 'scatter(size=...) instead' instead.

/home/alberto/miniconda3/envs/mint/l

In [86]:
p = hl.plot.qq(gwas.p_value)
show(p)

/home/alberto/miniconda3/envs/mint/lib/python3.10/site-packages/hail/plot/plots.py:930: BokehDeprecationWarning:

'circle() method with size value' was deprecated in Bokeh 3.4.0 and will be removed, use 'scatter(size=...) instead' instead.



The observed p-values drift away from the expectation immediately. Either every SNP in our dataset is causally linked to caffeine consumption (unlikely), or there’s a confounder.

The solution is to include ancestry as a covariate in our regression.

The linear_regression_rows function can also take column fields to use as covariates. We already annotated our samples with reported ancestry, but it is good to be skeptical of these labels due to human error. Genomes don’t have that problem! Instead of using reported ancestry, we will use genetic ancestry by including computed principal components in our model.

In [103]:
mt = hl.read_matrix_table('data/1kg_PCA.mt')
mt = hl.variant_qc(mt)

In [104]:
mt = mt.filter_rows(mt.variant_qc.AF[1] > 0.01)
mt = mt.filter_rows(mt.variant_qc.p_value_hwe > 1e-6)

In [105]:
gwas = hl.linear_regression_rows(
    y=mt.pheno.CaffeineConsumption,
    x=mt.GT.n_alt_alleles(),
    covariates=[1.0, mt.pheno.isFemale, mt.scores[0], mt.scores[1], mt.scores[2]])

In [106]:
p = hl.plot.qq(gwas.p_value)
show(p)

/home/alberto/miniconda3/envs/mint/lib/python3.10/site-packages/hail/plot/plots.py:930: BokehDeprecationWarning:

'circle() method with size value' was deprecated in Bokeh 3.4.0 and will be removed, use 'scatter(size=...) instead' instead.



In [107]:
p = hl.plot.manhattan(gwas.p_value)
show(p)

/home/alberto/miniconda3/envs/mint/lib/python3.10/site-packages/hail/plot/plots.py:980: BokehDeprecationWarning:

'circle() method with size value' was deprecated in Bokeh 3.4.0 and will be removed, use 'scatter(size=...) instead' instead.

/home/alberto/miniconda3/envs/mint/lib/python3.10/site-packages/hail/plot/plots.py:980: BokehDeprecationWarning:

'circle() method with size value' was deprecated in Bokeh 3.4.0 and will be removed, use 'scatter(size=...) instead' instead.

/home/alberto/miniconda3/envs/mint/lib/python3.10/site-packages/hail/plot/plots.py:980: BokehDeprecationWarning:

'circle() method with size value' was deprecated in Bokeh 3.4.0 and will be removed, use 'scatter(size=...) instead' instead.

/home/alberto/miniconda3/envs/mint/lib/python3.10/site-packages/hail/plot/plots.py:980: BokehDeprecationWarning:

'circle() method with size value' was deprecated in Bokeh 3.4.0 and will be removed, use 'scatter(size=...) instead' instead.

/home/alberto/miniconda3/envs/mint/l

Example of aggregation

In [100]:
entries = mt.entries()
results = (entries.group_by(pop = entries.pheno.SuperPopulation, chromosome = entries.locus.contig)
      .aggregate(n_het = hl.agg.count_where(entries.GT.is_het())))
results.show()

,,
pop,chromosome,n_het
str,str,int64
"""AFR""","""1""",16357
"""AFR""","""10""",10495
"""AFR""","""11""",10188
"""AFR""","""12""",10327
"""AFR""","""13""",6876
"""AFR""","""14""",6401
"""AFR""","""15""",5802
"""AFR""","""16""",7062
